In [1]:
from sklearn import datasets, svm
iris=datasets.load_iris()
iris.data[23:27]

array([[5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4]])

In [2]:
import pandas as pd
df=pd.DataFrame(iris.data, columns=iris.feature_names)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [3]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [4]:
df['flower']=iris.target
df['flower']=df['flower'].apply(lambda x: iris.target_names[x])
df[98:102]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
98,5.1,2.5,3.0,1.1,versicolor
99,5.7,2.8,4.1,1.3,versicolor
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.25)

In [6]:
sv=svm.SVC(kernel='rbf', C=30, gamma='auto')
sv.fit(X_train, y_train)
sv.score(X_test, y_test)

0.9210526315789473

In [7]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'), iris.data, iris.target, cv=4)

array([1.        , 0.94736842, 0.94594595, 1.        ])

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), iris.data, iris.target, cv=4)

array([0.97368421, 0.94736842, 0.97297297, 1.        ])

In [9]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='linear', C=20, gamma='auto'), iris.data, iris.target, cv=4)

array([1.        , 0.94736842, 0.97297297, 1.        ])

In [10]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='poly', C=10, gamma='auto'), iris.data, iris.target, cv=4)

array([1.        , 0.97368421, 0.94594595, 1.        ])

Instead of declaring all these variables individually, we can proceed with a function as follows below.

In [11]:
import numpy as np
kernels=['rbf', 'linear']
c_val=[5, 10, 20]
average_score={}

for kval in kernels:
    for cval in c_val:
        cv_scores=cross_val_score(svm.SVC(kernel=kval, C=cval, gamma='auto'), iris.data, iris.target, cv=4)
        average_score[kval + ' ' + str(cval)]=np.average(cv_scores)

average_score

{'rbf 5': 0.9735064011379801,
 'rbf 10': 0.9735064011379801,
 'rbf 20': 0.9601706970128023,
 'linear 5': 0.9800853485064012,
 'linear 10': 0.9733285917496444,
 'linear 20': 0.9800853485064012}

Using this loop, we can get an idea of the most suitable variables to select.

Another method is to use the `GridSearchCV` class from `sklearn.model_selection`. La classe `GridSearchCV` de la bibliothèque `sklearn.model_selection` est utilisée pour effectuer une recherche sur une grille d'hyperparamètres afin de trouver `les meilleurs paramètres` pour un modèle donné. Son rôle principal est de trouver la `combinaison d'hyperparamètres` qui maximise les performances d'un modèle donné sur un ensemble de données de validation croisée.

In [12]:
from sklearn.model_selection import GridSearchCV
gscv=GridSearchCV(svm.SVC(gamma='auto'),
                  {
                      'C':[5, 10, 20],
                      'kernel':['linear', 'rbf']
                  }, cv=4, return_train_score=False)
gscv.fit(X_train, y_train)
gscv.cv_results_

{'mean_fit_time': array([0.00074607, 0.00099373, 0.00075334, 0.00074768, 0.00075138,
        0.0005033 ]),
 'std_fit_time': array([4.30789101e-04, 5.24249874e-06, 4.34958821e-04, 4.31679240e-04,
        4.33825339e-04, 5.03325366e-04]),
 'mean_score_time': array([0.00075537, 0.00100487, 0.00049919, 0.00075382, 0.0005011 ,
        0.00049716]),
 'std_score_time': array([4.36217215e-04, 5.80923637e-06, 4.99194088e-04, 4.35243038e-04,
        5.01107032e-04, 4.97179499e-04]),
 'param_C': masked_array(data=[5, 5, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 5, 'kernel': 'linear'},
  {'C': 5, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 20, 'ker

In [13]:
cv_df=pd.DataFrame(gscv.cv_results_)
cv_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000746,0.000431,0.000755,0.000436,5,linear,"{'C': 5, 'kernel': 'linear'}",1.0,0.928571,1.000000,1.0,0.982143,0.030929,1
1,0.000994,0.000005,0.001005,0.000006,5,rbf,"{'C': 5, 'kernel': 'rbf'}",1.0,0.892857,1.000000,1.0,0.973214,0.046394,3
2,0.000753,0.000435,0.000499,0.000499,10,linear,"{'C': 10, 'kernel': 'linear'}",1.0,0.928571,1.000000,1.0,0.982143,0.030929,1
3,0.000748,0.000432,0.000754,0.000435,10,rbf,"{'C': 10, 'kernel': 'rbf'}",1.0,0.892857,1.000000,1.0,0.973214,0.046394,3
4,0.000751,0.000434,0.000501,0.000501,20,linear,"{'C': 20, 'kernel': 'linear'}",1.0,0.928571,0.964286,1.0,0.973214,0.029613,3
5,0.000503,0.000503,0.000497,0.000497,20,rbf,"{'C': 20, 'kernel': 'rbf'}",1.0,0.892857,1.000000,1.0,0.973214,0.046394,3


In [14]:
cv_df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,5,linear,0.982143
1,5,rbf,0.973214
2,10,linear,0.982143
3,10,rbf,0.973214
4,20,linear,0.973214
5,20,rbf,0.973214


In [15]:
gscv.best_score_    # pour voir le meilleur score(To display the best score)

0.9821428571428572

In [16]:
gscv.best_estimator_  # pour afficher la meilleure combinaison (To display the best combination)

SVC(C=5, gamma='auto', kernel='linear')

Another method is to use the `RandomizedSearchCV` class from `sklearn.model_selection`

In [17]:
from sklearn.model_selection import RandomizedSearchCV
rd=RandomizedSearchCV(svm.SVC(gamma='auto'),
                  {
                      'C':[5, 10, 20],
                      'kernel':['linear', 'rbf']
                  }, cv=4, return_train_score=False, n_iter=2) # n_iter  spécifie le nombre d'itérations à effectuer dans la recherche aléatoire.
rd.fit(X_train, y_train)
pd.DataFrame(rd.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,5,rbf,0.973214
1,20,rbf,0.973214


difference entre `RandomizedSearchCV` et `GridSearchCV` 
`RandomizedSearchCV` : Explore une grille d'hyperparamètres en sélectionnant aléatoirement un nombre fixe de combinaisons d'hyperparamètres à tester. (n_iter)
`GridSearchCV` : Effectue une recherche exhaustive sur une grille spécifiée d'hyperparamètres, testant chaque combinaison possible.

`RandomizedSearchCV`: Explores a grid of hyperparameters by randomly selecting a fixed number of hyperparameter combinations to test (n_iter).
`GridSearchCV`: Performs an exhaustive search over a specified grid of hyperparameters, testing every possible combination.

To choose the `Best Model`

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [19]:
model_params = {
    'lr':{
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params':{ 'C':[1,5,10]}
    },
    'sv':{
        'model':SVC(gamma='auto'),
        'params':{
            'kernel':['rbf', 'linear'],
            'C':[5,10,20]
        }
    },
    'rfc':{
        'model': RandomForestClassifier(criterion='gini'),
        'params': {
            'n_estimators': [1,5,10]
        }
    }
}

In [20]:
scores=[]

for model_name, mp in model_params.items():
    clf=GridSearchCV(mp['model'], mp['params'], cv=4, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append(
        {'model':model_name,
         'best_score': clf.best_score_,
         'best_params': clf.best_params_
        }
    )

In [21]:
data=pd.DataFrame(scores, columns=['model','best_score','best_params'])
data

,model,best_score,best_params
0,lr,0.966394,{'C': 10}
1,sv,0.980085,"{'C': 5, 'kernel': 'linear'}"
2,rfc,0.953236,{'n_estimators': 10}


The best model is `Support Vector Machine` with this parameters: `'C': 5, 'kernel': 'linear'`